In [111]:
"""Importing header files"""
import os
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from os.path import exists
from os.path import join
from subprocess import call
from urllib import urlretrieve
from urlparse import urljoin
from Tkinter import *

In [112]:
def download(year, link) :
    """To download the data pdf from website and convert it to .txt format"""
    
    pdfpath = join("pdftext",year+".pdf")
    # Create local path for pdf
    if not exists(pdfpath) :
        urlretrieve(link, pdfpath) #Copy a network object denoted by a URL to a local file, if necessary.
        
    txtpath = pdfpath.replace(".pdf",".txt")
    if not exists(txtpath) :
        call(["pdftotext" ,"-layout", pdfpath, txtpath])

        
download("2014","http://eci.nic.in/eci_main/archiveofge2014/33%20-%20Constituency%20wise%20detailed%20result.pdf")

In [113]:
"""Declarations"""
field = {"2014.txt": ['NAME', 'SEX', 'AGE', 'CATEGORY', 'PARTY', 'SYMBOL', 'GENERAL VOTES', 'POSTAL VOTES', 'VOTES', '% ELECTORS', '% VOTES']}
dictionary = {}

In [114]:
def dataclean (txtfile) :
    re_state = re.compile("^[A-Z][A-Za-z &]+$")
    re_constielectors = re.compile("CONSTITUENCY *: *\d+ *\. *(.*)")
    #re_details = re.compile("\d *[A-Z a-z \.]*")
    
    fieldlist = field[txtfile]
    constituency = None
    electors = None
    state = None
    output = []
    for line in open("pdftext/"+txtfile) :
        match = re_state.match(line)
        if match and len(line)<30:
            state = match.group()
            continue
        
            
        match = re_constielectors.match(line)
        if match :
            constielectors = match.groups()[0].split()
            #constielectors_list = constielectors.split()
            constituency = constielectors[0]
            electors = constielectors[-2]
            #print constituency,electors
            continue
            
            
        details = re.split('  +',line.strip())
        if len(details) == len(fieldlist) :    
            #zipped = zip(fieldlist , details)
            dictionary = dict(zip(fieldlist,details))
            dictionary['NAME'] = ' '.join(dictionary['NAME'].split()[1:])
        else :
            continue
        
        dictionary['STATE'] = state
        dictionary['CONSTITUENCY'] = constituency
        dictionary['YEAR'] = txtfile.split('.')[0]
        output.append(dictionary)
        
    
    df = pd.DataFrame(output).set_index(['STATE','CONSTITUENCY','YEAR'])
    if '%' in df:
        del df['%']
    df.reset_index()
    df.to_csv("2014.csv")
    return output
    


In [171]:
def plot(dictionary) :
    X=['STATE', 'CONSTITUENCY', 'AGE', 'CATEGORY', 'PARTY', 'SEX']
    Y=['% ELECTORS', '% VOTES', 'GENERAL VOTES', 'POSTAL VOTES', 'VOTES']
    x=''
    y=''
    print "Let's do some simple visualization !"
    print ""
    print "X: STATE, CONSTITUENCY, AGE, CATEGORY, PARTY, SEX"
    print "Y: % ELECTORS, % VOTES, GENERAL VOTES, POSTAL VOTES, VOTES"
    
    while x not in X:
        x = raw_input("Give input for X-axis")
        x = x.upper()
    while y not in Y:
        y = raw_input("Give input for Y-axis")
        y = y.upper()
    
    df= pd.read_csv('2014.csv')
    df1= df[[x,y]].groupby(x, as_index=False).sum()
    
    ax= df1[[x,y]].plot(x=x, y=y, kind='bar',figsize=(20, 15), legend=False)
    plt.xlabel(x)
    plt.ylabel(y)
    plt.show()

In [172]:
plot(dataclean("2014.txt"))

Let's do some simple visualization !

X: STATE, CONSTITUENCY, AGE, CATEGORY, PARTY, SEX
Y: % ELECTORS, % VOTES, GENERAL VOTES, POSTAL VOTES, VOTES
Give input for X-axissex
Give input for Y-axisvotes
